In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import squarify
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv(r"C:\Users\lenovo\OneDrive\Desktop\HRUTUJA\Imarticus\Advanced ML\RFM\Retail_Data_Transactions.csv")
df.head()

,customer_id,trans_date,tran_amount
0,CS5295,11-Feb-17,35
1,CS4768,15-Mar-19,39
2,CS2122,26-Feb-17,52
3,CS1217,16-Nov-15,99
4,CS1850,20-Nov-17,78


In [4]:
df['trans_date']=pd.to_datetime(df['trans_date'])
df.head()

,customer_id,trans_date,tran_amount
0,CS5295,2017-02-11,35
1,CS4768,2019-03-15,39
2,CS2122,2017-02-26,52
3,CS1217,2015-11-16,99
4,CS1850,2017-11-20,78


In [8]:
print('No. of Customers :',len(list(df['customer_id'].unique())))
print('Rows :' , df.shape[0] ,' | ','Columns :' ,df.shape[1])
print('Transactions don\'t have a customer id :',(df[df.customer_id.isnull()].shape[0]))
print('Transactions TimeFrame Duration : Starting ',df['trans_date'].min(),' |' ,'Ending',df['trans_date'].max())

No. of Customers : 6889
Rows : 125000  |  Columns : 3
Transactions don't have a customer id : 0
Transactions TimeFrame Duration : Starting  2015-05-16 00:00:00  | Ending 2019-03-16 00:00:00


In [10]:
df=df.dropna()

In [12]:
print('Rows :' , df.shape[0] ,' | ','Columns :' ,df.shape[1])

Rows : 125000  |  Columns : 3


In [14]:
snapshot_date = df['trans_date'].max() + timedelta(days=1)
print(snapshot_date)

2019-03-17 00:00:00


In [22]:
data_process = df.groupby(['customer_id']).agg({
        'trans_date': lambda x: (snapshot_date - x.max()).days,
        'customer_id': 'count',
        'tran_amount': 'sum'})
data_process.head()

,trans_date,customer_id,tran_amount
customer_id,,,
CS1112,62,15,1012
CS1113,36,20,1490
CS1114,33,19,1432
CS1115,12,22,1659
CS1116,204,13,857


In [24]:
data_process.rename(columns={'trans_date': 'Recency',
                         'customer_id': 'Frequency',
                         'tran_amount': 'MonetaryValue'}, inplace=True)

data_process

,Recency,Frequency,MonetaryValue
customer_id,,,
CS1112,62,15,1012
CS1113,36,20,1490
CS1114,33,19,1432
CS1115,12,22,1659
CS1116,204,13,857
...,...,...,...
CS8996,98,13,582
CS8997,262,14,543
CS8998,85,13,624


In [26]:
data_process.head()

,Recency,Frequency,MonetaryValue
customer_id,,,
CS1112,62,15,1012
CS1113,36,20,1490
CS1114,33,19,1432
CS1115,12,22,1659
CS1116,204,13,857


In [27]:
r_labels = range(4, 0, -1)  # 4 , 3 , 2 , 1 
f_labels = range(1, 5)      # 1 , 2 ,3 ,4 
m_labels = range(1, 5)  # 1 , 2 ,3 ,4
 
r_groups = pd.qcut(data_process['Recency'], q=4, labels=r_labels)

f_groups = pd.qcut(data_process['Frequency'], q=4, labels=f_labels)

m_groups = pd.qcut(data_process['MonetaryValue'], q=4, labels=m_labels)

data_process = data_process.assign(Recency_Rank = r_groups.values,
                                   Frequency_Rank = f_groups.values,
                                   Moentary_Rank = m_groups.values)


data_process.head()

,Recency,Frequency,MonetaryValue,Recency_Rank,Frequency_Rank,Moentary_Rank
customer_id,,,,,,
CS1112,62,15,1012,2,2,2
CS1113,36,20,1490,3,3,3
CS1114,33,19,1432,3,3,3
CS1115,12,22,1659,4,3,4
CS1116,204,13,857,1,1,2


In [30]:
data_process['RFM_Segment_Concat'] = data_process.Recency_Rank.astype(str) + data_process.Frequency_Rank.astype(str) + data_process.Moentary_Rank.astype(str)
rfm = data_process
rfm.head()

,Recency,Frequency,MonetaryValue,Recency_Rank,Frequency_Rank,Moentary_Rank,RFM_Segment_Concat
customer_id,,,,,,,
CS1112,62,15,1012,2,2,2,222
CS1113,36,20,1490,3,3,3,333
CS1114,33,19,1432,3,3,3,333
CS1115,12,22,1659,4,3,4,434
CS1116,204,13,857,1,1,2,112


In [32]:
rfm_count_unique = rfm.groupby('RFM_Segment_Concat')['RFM_Segment_Concat'].nunique()
print("Combination of Ranks  :", rfm_count_unique.sum())

Combination of Ranks  : 48


In [34]:
# Calculate RFM_Score
rfm['RFM_Score'] = rfm[['Recency_Rank','Frequency_Rank','Moentary_Rank']].sum(axis=1)
rfm.head()

,Recency,Frequency,MonetaryValue,Recency_Rank,Frequency_Rank,Moentary_Rank,RFM_Segment_Concat,RFM_Score
customer_id,,,,,,,,
CS1112,62,15,1012,2,2,2,222,6
CS1113,36,20,1490,3,3,3,333,9
CS1114,33,19,1432,3,3,3,333,9
CS1115,12,22,1659,4,3,4,434,11
CS1116,204,13,857,1,1,2,112,4


In [36]:
def rfm_level(df):
    if df['RFM_Score'] >= 10:
        return 'Loyal / Potential / Can\'t Loose Them'
    elif ((df['RFM_Score'] >= 5) and (df['RFM_Score'] < 10)):
        return 'Champions / Promising '
    elif ((df['RFM_Score'] >= 4) and (df['RFM_Score'] < 5)):
        return 'Needs Attention'
    else:
        return 'Require Activation'

In [38]:
rfm['Customer_Segment'] = rfm.apply(rfm_level, axis=1)
rfm.head()

,Recency,Frequency,MonetaryValue,Recency_Rank,Frequency_Rank,Moentary_Rank,RFM_Segment_Concat,RFM_Score,Customer_Segment
customer_id,,,,,,,,,
CS1112,62,15,1012,2,2,2,222,6,Champions / Promising
CS1113,36,20,1490,3,3,3,333,9,Champions / Promising
CS1114,33,19,1432,3,3,3,333,9,Champions / Promising
CS1115,12,22,1659,4,3,4,434,11,Loyal / Potential / Can't Loose Them
CS1116,204,13,857,1,1,2,112,4,Needs Attention
